<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [3]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use ABS(MOD()) on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [4]:
df = bigquery.Client().query("SELECT COUNT(weight_pounds) AS num_babies FROM (" + query + ") where abs(mod(hashmonth,10))<8 \
                                                                                                    and rand() < 0.0005").to_dataframe()
df

,num_babies
0,13412


As this number is close to what we want in the trainig dataset. We can run the query and get the dataset.

In [5]:
traindf = bigquery.Client().query("SELECT * FROM (" + query + ") where abs(mod(hashmonth,10))<8 and rand() < 0.0005").to_dataframe()
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,8.013803,True,30,1,41.0,3095933535584005890
1,7.187070,True,25,1,38.0,3095933535584005890
2,7.142977,True,16,1,40.0,3095933535584005890
3,5.687926,False,22,1,40.0,3095933535584005890
4,6.375769,True,22,1,37.0,3095933535584005890


In [6]:
evaldf = bigquery.Client().query("SELECT * FROM (" + query + ") where abs(mod(hashmonth,10))>=8 and rand() < 0.0005").to_dataframe()
evaldf.shape

(3152, 6)

## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [7]:
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13604.000000,13618.000000,13618.000000,13513.000000,1.361800e+04
mean,7.212893,27.431488,1.037083,38.599497,-4.729820e+16
std,1.310543,6.190013,0.201389,2.562026,5.202422e+18
min,0.661387,13.000000,1.000000,19.000000,-9.183606e+18
25%,6.563162,22.000000,1.000000,38.000000,-4.979698e+18
50%,7.312733,27.000000,1.000000,39.000000,-5.245312e+17
75%,8.024826,32.000000,1.000000,40.000000,3.765901e+18
max,13.375445,48.000000,4.000000,47.000000,8.599690e+18


I will be writing a preprocess function which I will be applying to both the traindf and evaldf

In [8]:
import pandas as pd

def preprocess(df):
    df = df[df.weight_pounds.notnull()]
    df = df[df.mother_age.notnull()]
    df = df[df.plurality.notnull()]
    df = df[df.gestation_weeks.notnull()]
    
    dic = {1:'Single(1)',2:'Twins(2)', 3:'Triplets(3)', 4:'Quadruplets(4)', 5:'Quintuplets(5)'}
    df['plurality'] = df['plurality'].map(dic)
    
    new_df = df.copy(deep=True)
    new_df['is_male'] = 'Unknown'
    
    return pd.concat([df,new_df])

traindf = preprocess(traindf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,8.013803,True,30,Single(1),41.0,3095933535584005890
1,7.187070,True,25,Single(1),38.0,3095933535584005890
2,7.142977,True,16,Single(1),40.0,3095933535584005890
3,5.687926,False,22,Single(1),40.0,3095933535584005890
4,6.375769,True,22,Single(1),37.0,3095933535584005890


In [9]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13613,7.043769,Unknown,24,Single(1),37.0,74931465496927487
13614,6.750554,Unknown,30,Single(1),39.0,74931465496927487
13615,4.250512,Unknown,23,Single(1),35.0,74931465496927487
13616,7.561856,Unknown,25,Single(1),39.0,74931465496927487
13617,8.249698,Unknown,29,Single(1),40.0,74931465496927487


In [10]:
evaldf = preprocess(evaldf)

## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [11]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [12]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   6258 eval.csv
  27002 train.csv
  33260 total
==> eval.csv <==
7.8043640748,False,32,Single(1),38.0,8904940584331855459
8.12623897732,True,27,Single(1),38.0,3572456083892950068
6.87621795178,True,30,Single(1),37.0,8904940584331855459
7.62578964258,True,33,Single(1),39.0,-2995620979373137889
7.31273323054,False,23,Single(1),40.0,7586701093880244808
6.75055446244,False,20,Single(1),38.0,-7146494315947640619
8.487797087,False,27,Single(1),39.0,-7146494315947640619
5.8753192823,False,27,Single(1),39.0,-260598435387740869
8.87801529074,True,26,Single(1),40.0,-2768780346491825728
6.9996768185,True,37,Single(1),39.0,-7146494315947640619

==> train.csv <==
8.0138032237,True,30,Single(1),41.0,3095933535584005890
7.1870697412,True,25,Single(1),38.0,3095933535584005890
7.1429772888,True,16,Single(1),40.0,3095933535584005890
5.6879263596,False,22,Single(1),40.0,3095933535584005890
6.37576861704,True,22,Single(1),37.0,3095933535584005890
8.1240343547,True,28,Single(1),42.0,3095933535584005890
8.

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License